# <center> <font size="+10"> <bold> ETL </bold> </font> </center>

In [12]:
#Import dependencies
import pandas as pd
from sqlalchemy import create_engine

# Extract

In [13]:
#Read csv obtained from Mexico's Government site
CP_CDMX_df = pd.read_csv("Resources/CodigosPostales.csv")
CP_CDMX_df.head()

,Código Postal,Estado,Municipio,Ciudad,Tipo de Asentamiento,Asentamiento,Clave de Oficina
0,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Lomas El Manto,9821
1,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Los Ángeles,9821
2,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Paraje San Juan,9821
3,9837,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,San Miguel 8va. Ampliación,9821
4,9838,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Plan de Iguala,9821


In [14]:
#Read csv obtained from Kaggle and last updated in October 2019.

Airbnb_CDMX_df = pd.read_csv("Resources/AirbnbCDMX.csv")
Airbnb_CDMX_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,14714,Private room c/ bathroom . Colonia Juarez.,57785,Diego,NaN,Cuauhtémoc,19.43035,-99.15511,Private room,478,2,0,NaN,NaN,2,327
1,22787,"Sunny suite w/ queen size bed, inside boutique...",87973,Diego,NaN,Cuauhtémoc,19.44076,-99.16324,Private room,1969,1,57,2019-05-27,0.52,9,355
2,33681,Couple of Rooms,145672,Edubiel,NaN,Tlalpan,19.27215,-99.21848,Private room,1740,1,0,NaN,NaN,1,365
3,35797,Villa Dante,153786,Dici,NaN,Cuajimalpa de Morelos,19.38399,-99.27335,Entire home/apt,3823,1,0,NaN,NaN,2,363
4,44616,CONDESA HAUS B&B,196253,Condesa Haus Bed & Breakfast CDMX,NaN,Cuauhtémoc,19.41006,-99.17645,Private room,1893,1,39,2019-05-02,0.42,10,334


In [15]:
#Read csv obtained from Kaggle and last updated in February 2017.

Starbucksdirectory_df = pd.read_csv("Resources/Starbucksdirectory.csv")
Starbucksdirectory_df.head()

,Brand,Store Number,Store Name,Ownership Type,Street Address,City,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude
0,Starbucks,47370-257954,"Meritxell, 96",Licensed,"Av. Meritxell, 96",Andorra la Vella,7,AD,AD500,376818720,GMT+1:00 Europe/Andorra,1.53,42.51
1,Starbucks,22331-212325,Ajman Drive Thru,Licensed,"1 Street 69, Al Jarf",Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.42
2,Starbucks,47089-256771,Dana Mall,Licensed,Sheikh Khalifa Bin Zayed St.,Ajman,AJ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,55.47,25.39
3,Starbucks,22126-218024,Twofour 54,Licensed,Al Salam Street,Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.38,24.48
4,Starbucks,17127-178586,Al Ain Tower,Licensed,"Khaldiya Area, Abu Dhabi Island",Abu Dhabi,AZ,AE,NaN,NaN,GMT+04:00 Asia/Dubai,54.54,24.51


# Transform

In [16]:
# Change table language to English

CP_CDXM_df_e = CP_CDMX_df.rename(columns = {'Código Postal':'Postcode',
                            'Estado':'State',
                            'Municipio':'Municipality',
                            'Ciudad':'City',
                           'Tipo de Asentamiento':'Settlement Type',
                           'Asentamiento':'Settlement',
                           'Clave de Oficina':'Office Code'})
CP_CDXM_df_e.head()

,Postcode,State,Municipality,City,Settlement Type,Settlement,Office Code
0,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Lomas El Manto,9821
1,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Los Ángeles,9821
2,9830,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Paraje San Juan,9821
3,9837,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,San Miguel 8va. Ampliación,9821
4,9838,Ciudad de México,Iztapalapa,Ciudad de México,Colonia,Plan de Iguala,9821


In [17]:
# Select only the columns we need
new_postal_codes = CP_CDXM_df_e[['Postcode', 'State', 'Municipality', 'City', 'Settlement']].copy()
new_postal_codes.head()

,Postcode,State,Municipality,City,Settlement
0,9830,Ciudad de México,Iztapalapa,Ciudad de México,Lomas El Manto
1,9830,Ciudad de México,Iztapalapa,Ciudad de México,Los Ángeles
2,9830,Ciudad de México,Iztapalapa,Ciudad de México,Paraje San Juan
3,9837,Ciudad de México,Iztapalapa,Ciudad de México,San Miguel 8va. Ampliación
4,9838,Ciudad de México,Iztapalapa,Ciudad de México,Plan de Iguala


In [18]:
#Select only the columns we need from Airbnb file.

new_Airbnb = Airbnb_CDMX_df[['name', 'neighbourhood', 'latitude', 'longitude', 'room_type']].copy()
new_Airbnb.head()

,name,neighbourhood,latitude,longitude,room_type
0,Private room c/ bathroom . Colonia Juarez.,Cuauhtémoc,19.43035,-99.15511,Private room
1,"Sunny suite w/ queen size bed, inside boutique...",Cuauhtémoc,19.44076,-99.16324,Private room
2,Couple of Rooms,Tlalpan,19.27215,-99.21848,Private room
3,Villa Dante,Cuajimalpa de Morelos,19.38399,-99.27335,Entire home/apt
4,CONDESA HAUS B&B,Cuauhtémoc,19.41006,-99.17645,Private room


In [19]:
#Select only the columns we need from Starbucks file.

new_Starbucks = Starbucksdirectory_df [['Store Name', 'Street Address', 'State/Province', 'Country', 'Postcode', 'Longitude', 'Latitude']].copy()
new_Starbucks.head()

,Store Name,Street Address,State/Province,Country,Postcode,Longitude,Latitude
0,"Meritxell, 96","Av. Meritxell, 96",7,AD,AD500,1.53,42.51
1,Ajman Drive Thru,"1 Street 69, Al Jarf",AJ,AE,NaN,55.47,25.42
2,Dana Mall,Sheikh Khalifa Bin Zayed St.,AJ,AE,NaN,55.47,25.39
3,Twofour 54,Al Salam Street,AZ,AE,NaN,54.38,24.48
4,Al Ain Tower,"Khaldiya Area, Abu Dhabi Island",AZ,AE,NaN,54.54,24.51


In [20]:
#Filter Starbucks data for Mexico City only

Starbucks_MX = new_Starbucks.loc [new_Starbucks['Country']== 'MX']
Starbucks_MX = new_Starbucks.loc [new_Starbucks['State/Province'] == 'DIF']
Starbucks_MX

,Store Name,Street Address,State/Province,Country,Postcode,Longitude,Latitude
9184,STIM,"de la Industria Militar 1296,, Lomas de Chamiz...",DIF,MX,5129,-99.26,19.39
9185,EMERSON,"Homero n.503, Col. Chapultepec Morales, Del. M...",DIF,MX,11570,-99.19,19.44
9186,Narvarte,"Palanque s/n, Col. Narvarte",DIF,MX,1030,-99.15,19.40
9187,Pasaje San Jeronimo,AV SAN JERONIMO 819 COL SAN JERONIMO,DIF,MX,10200,-99.23,19.32
9188,RENATO LEDUC,RENATO LEDUC 224 COL TORIELLO GUERRA,DIF,MX,14050,-99.16,19.30
...,...,...,...,...,...,...,...
9408,Forum Buenavista II,"Eje 1 Nte Mosqueta 259 Buenavista, Eje 1 Nte M...",DIF,MX,6350,-99.15,19.45
9409,"Shiller, Masaryk 111","Shiller esquina Masaryk 111, Polanco, Shiller ...",DIF,MX,11550,-99.19,19.43
9410,Axomiatla II,"Calz. Las Aguilas 1953 Col. Axomiatla, Del. Al...",DIF,MX,11520,-99.25,19.34
9411,Plaza Basica,"Av. Leyes de Reforma esq. Baratillo, Col. Ejid...",DIF,MX,9040,-99.09,19.38


# Load

In [21]:
#Export the 3 tables in csv's to use them in SQL/ PgAdmin

Starbucks_MX.to_csv("Resources/Starbucks_cleaned.csv", index=True, header=True)
new_postal_codes.to_csv ("Resources/Postcodes_cleaned.csv", index=True, header=True)
new_Airbnb.to_csv ("Resources/Airbnb_cleaned.csv", index=True, header=True)